In [4]:
import pandas as pd
import numpy as np
import geojson
import geopandas
from pathlib import Path
import requests
from math import floor
import json
from datetime import datetime, timedelta
import os
import xml.etree.ElementTree as ET
import locale
import zipfile
import tempfile
import glob

# DATA_PATH expects a working structure with mets10 data generated and city subfolders
# ├── loop_counters
# │   ├── berlin
# │   │   ├── downloads
# │   │   └── speed
# │   ├── london
# │   │   ├── downloads
# │   │   └── speed
# │   └── madrid
# │       ├── all
# │       └── downloads
# └── release20221026_residential_unclassified
#     ├── 2021
#     │   ├── road_graph
#     │   │   └── berlin
#     │   └── speed_classes
#     │       └── berlin
#     └── 2022
#         ├── road_graph
#         │   ├── london
#         │   └── madrid
#         └── speed_classes
#             ├── london
#             └── madrid
DATA_PATH = Path('/private/data/mets10') 
BERLIN_PATH = DATA_PATH / 'loop_counters' / 'berlin'
LONDON_PATH = DATA_PATH / 'loop_counters' / 'london'
MADRID_PATH = DATA_PATH / 'loop_counters' / 'madrid'

/usr/local/Caskroom/miniconda/base/envs/t4c/lib/python3.8/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Berlin

Raw loop counter data was downloaded from
https://api.viz.berlin.de/daten/verkehrsdetektion/teu_standorte.json and https://api.viz.berlin.de/daten/verkehrsdetektion?path=2020%2FDetektoren+%28einzelne+Fahrspur%29%2F

In [3]:
berlin_locations_df = pd.read_excel(BERLIN_PATH / 'downloads' / 'Stammdaten_Verkehrsdetektion_2022_07_20.xlsx')
berlin_locations_df = berlin_locations_df[[
    'DET_NAME_NEU', 'DET_ID15', 'RICHTUNG', 'SPUR', 'LÄNGE (WGS84)', 'BREITE (WGS84)']]
berlin_locations_df = berlin_locations_df.rename(columns={
    'DET_NAME_NEU': 'detname', 'DET_ID15': 'detid_15', 'LÄNGE (WGS84)': 'lon', 'BREITE (WGS84)': 'lat',
    'SPUR': 'lane', 'RICHTUNG': 'heading'})
berlin_locations_df = berlin_locations_df.replace({'heading': {
    'Nord': 0, 'Nordost': 45, 'Ost': 90, 'Südost': 135,
    'Süd': 180, 'Südwest': 225, 'West': 270, 'Nordwest': 315
}})
berlin_locations_df = berlin_locations_df.groupby(by=['detname', 'detid_15', 'heading', 'lane', 'lon', 'lat']).count()
berlin_locations_df = berlin_locations_df.reset_index()
berlin_locations_df

,detname,detid_15,heading,lane,lon,lat
0,TE001_Det_HF1,100101010000167,225,HF_R,13.192578,52.433868
1,TE001_Det_HF2,100101010000268,225,HF_2vR,13.192578,52.433868
2,TE002_Det_HF1,100101010000369,45,HF_R,13.192747,52.433813
3,TE002_Det_HF2,100101010000470,45,HF_2vR,13.192747,52.433813
4,TE004_Det_HF1,100101010000874,180,HF_R,13.261301,52.436642
...,...,...,...,...,...,...
543,TE583_Det_HF2,100101010097975,0,HF_2vR,13.384196,52.457440
544,TE592_Det_HF1,100101010099692,180,HF_R,13.301719,52.509232
545,TE592_Det_HF2,100101010099793,180,HF_2vR,13.301719,52.509232
546,TE593_Det_HF1,100101010099894,0,HF_R,13.302183,52.508531


In [4]:
berlin_locations_df[berlin_locations_df['detid_15'].duplicated()]

,detname,detid_15,heading,lane,lon,lat


In [5]:
def get_counters_merged(year, month, locations_df):
    df = pd.read_csv(BERLIN_PATH / 'downloads' / f'det_val_hr_{year:04d}_{month:02d}.csv.gz', sep=';',
                     compression='gzip')
    print(f'Loaded {len(df)} rows with {len(df["detid_15"].unique())} unique counters')
    df = df.merge(locations_df, on=['detid_15'], how='left')
    df = df.rename(columns={'detid_15': 'id', 'detname': 'name'})
    df['time_bin'] = [f'{d[6:]}-{d[3:5]}-{d[:2]} {h:02d}:00' for d, h in zip(df['tag'], df['stunde'])]
    df.to_parquet(BERLIN_PATH / 'speed' / f'counters_{year:04d}-{month:02d}.parquet', compression="snappy")
    return df
    
berlin202107_df = get_counters_merged(2021, 7, berlin_locations_df)
berlin202107_df

Loaded 294180 rows with 469 unique counters


,id,tag,stunde,qualitaet,q_kfz_det_hr,v_kfz_det_hr,q_pkw_det_hr,v_pkw_det_hr,q_lkw_det_hr,v_lkw_det_hr,name,heading,lane,lon,lat,time_bin
0,100101010001076,01.07.2021,3,0.75,36,39,7,48,29,36,TE005_Det_HF1,270,HF_R,13.263106,52.435112,2021-07-01 03:00
1,100101010001076,01.07.2021,4,0.92,63,60,12,54,51,61,TE005_Det_HF1,270,HF_R,13.263106,52.435112,2021-07-01 04:00
2,100101010001076,01.07.2021,5,1.00,215,65,42,64,173,65,TE005_Det_HF1,270,HF_R,13.263106,52.435112,2021-07-01 05:00
3,100101010001076,01.07.2021,6,0.83,462,76,119,105,343,66,TE005_Det_HF1,270,HF_R,13.263106,52.435112,2021-07-01 06:00
4,100101010001076,01.07.2021,7,1.00,810,90,119,100,691,88,TE005_Det_HF1,270,HF_R,13.263106,52.435112,2021-07-01 07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294175,100101010093329,31.07.2021,19,1.00,263,30,254,30,9,32,TE559_Det_HF1,90,HF_R,13.387089,52.531362,2021-07-31 19:00
294176,100101010093329,31.07.2021,20,1.00,195,30,191,30,4,33,TE559_Det_HF1,90,HF_R,13.387089,52.531362,2021-07-31 20:00
294177,100101010093329,31.07.2021,21,1.00,161,29,159,30,2,33,TE559_Det_HF1,90,HF_R,13.387089,52.531362,2021-07-31 21:00
294178,100101010093329,31.07.2021,22,1.00,132,32,126,32,6,33,TE559_Det_HF1,90,HF_R,13.387089,52.531362,2021-07-31 22:00


In [6]:
berlin201906_df = get_counters_merged(2019, 6, berlin_locations_df)
berlin201906_df

Loaded 373739 rows with 546 unique counters


,id,tag,stunde,qualitaet,q_kfz_det_hr,v_kfz_det_hr,q_pkw_det_hr,v_pkw_det_hr,q_lkw_det_hr,v_lkw_det_hr,name,heading,lane,lon,lat,time_bin
0,100101010000167,01.06.2019,0,1.0,227,82.8,207,83.9,20,71.6,TE001_Det_HF1,225,HF_R,13.192578,52.433868,2019-06-01 00:00
1,100101010000167,01.06.2019,1,1.0,131,78.4,115,80.5,16,63.7,TE001_Det_HF1,225,HF_R,13.192578,52.433868,2019-06-01 01:00
2,100101010000167,01.06.2019,2,1.0,94,77.0,74,80.6,20,63.3,TE001_Det_HF1,225,HF_R,13.192578,52.433868,2019-06-01 02:00
3,100101010000167,01.06.2019,3,1.0,83,76.4,68,76.2,15,77.3,TE001_Det_HF1,225,HF_R,13.192578,52.433868,2019-06-01 03:00
4,100101010000167,01.06.2019,4,1.0,131,76.9,118,78.8,13,59.9,TE001_Det_HF1,225,HF_R,13.192578,52.433868,2019-06-01 04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373734,100101010099995,30.06.2019,19,1.0,191,43.4,189,43.4,2,42.0,TE593_Det_HF2,0,HF_2vR,13.302183,52.508531,2019-06-30 19:00
373735,100101010099995,30.06.2019,20,1.0,219,42.2,216,42.4,3,27.3,TE593_Det_HF2,0,HF_2vR,13.302183,52.508531,2019-06-30 20:00
373736,100101010099995,30.06.2019,21,1.0,265,39.6,257,39.9,8,29.5,TE593_Det_HF2,0,HF_2vR,13.302183,52.508531,2019-06-30 21:00
373737,100101010099995,30.06.2019,22,1.0,158,43.2,157,43.3,1,42.0,TE593_Det_HF2,0,HF_2vR,13.302183,52.508531,2019-06-30 22:00


# London

Scraping counter data from the Highways England WebTRIS API https://webtris.highwaysengland.co.uk/

In [7]:
WEBTRIS = 'https://webtris.highwaysengland.co.uk/api/v1'
LONDON_BBOX = [5120500, 5170000, -36900, 6700]

def save_json_rows(data, file_name):
    if not data:
        return
    with open(LONDON_PATH / 'downloads' / f'{file_name}.json', 'w') as f:
        for row in data:
            json.dump(row, f)
            f.write('\n')

def get_webtris_sites(bbox):
    min_lat, max_lat, min_lon, max_lon = tuple(bbox)
    url = f'{WEBTRIS}/sites'
    req = requests.get(url)
    req_json = req.json()
    print(req_json['row_count'])
    for site in req_json['sites']:
        lat_bin = int(floor(site['Latitude'] * 1e3) / 1e3 * 1e5)
        lon_bin = int(floor(site['Longitude'] * 1e3) / 1e3 * 1e5)
        if lat_bin >= min_lat and lat_bin <= max_lat:
            site['lat_bin'] = lat_bin
            site['lon_bin'] = lon_bin
            yield site

def time_ceil(time, delta):
    mod = (time - datetime(1970, 1, 1)) % delta
    if mod:
        return time + (delta - mod)
    return time

def time_bin_format(time):
    return time.strftime('%Y-%m-%d %H:%M')

def get_time_bins(datetime_str):
    time_bins = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
    time_bins = time_ceil(time_bins, timedelta(minutes=5))
    return [
        time_bin_format(time_bins-timedelta(minutes=10)),
        time_bin_format(time_bins-timedelta(minutes=5)),
        time_bin_format(time_bins)
    ]

def join_site_info(data, sites_dict):
    for row in data:
        site = sites_dict[row['Site Name']]
        volume = row['Total Volume']
        if volume:
            volume = int(volume)
        else:
            volume = 0
        speed = row['Avg mph']
        if speed:
            speed = float(speed)*1.60934
        else:
            speed = 0
        time_bins = get_time_bins(f"{row['Report Date'][:10]} {row['Time Period Ending']}")
        yield {
            'id': int(site['Id']), 'name': row['Site Name'],
            'lat_bin': site['lat_bin'], 'lon_bin': site['lon_bin'],
            'lat': site['Latitude'], 'lon': site['Longitude'],
            'time_bins': time_bins,
            'volume': volume, 'speed': speed
        }

def get_chunk(id_chunk, date_from, date_to):
    date_from = datetime.strptime(date_from, '%Y-%m-%d').strftime('%d%m%Y')
    date_to = datetime.strptime(date_to, '%Y-%m-%d').strftime('%d%m%Y')
    req_ids_urlenc = ','.join(id_chunk)
    url = f'{WEBTRIS}/reports/{date_from}/to/{date_to}/Daily?sites={req_ids_urlenc}&page=1&page_size=10000'
    while True:
        print(url)
        req = requests.get(url)
        if req.status_code == 204:
            return
        req_json = req.json()
        yield from req_json['Rows']
        header = req_json['Header']
        print(header)
        if 'links' in header and len(header['links']) > 0 and header['links'][-1]['rel'] == 'nextPage':
            url =  header['links'][-1]['href']
            continue
        else:
            break

In [8]:
webtris_sites = list(get_webtris_sites(LONDON_BBOX))
save_json_rows(webtris_sites, 'webtris_sites')
print(len(webtris_sites))
webtris_sites[:2]

19355
4256


[{'Id': '1',
  'Name': 'MIDAS site at M4/2295A2 priority 1 on link 105009001; GPS Ref: 502816;178156; Westbound',
  'Description': 'M4/2295A2',
  'Longitude': -0.520379557723297,
  'Latitude': 51.4930115367112,
  'Status': 'Inactive',
  'lat_bin': 5149300,
  'lon_bin': -52100},
 {'Id': '5',
  'Name': 'MIDAS site at M25/5764B priority 1 on link 199056702; GPS Ref: 558308;188775; Anti-clockwise',
  'Description': 'M25/5764B',
  'Longitude': 0.283161593410359,
  'Latitude': 51.5756168053165,
  'Status': 'Active',
  'lat_bin': 5157500,
  'lon_bin': 28299}]

In [9]:
def download_webtris_chunks(webtris_sites, date_from, date_to, chunk_size=20):
    sites_dict = { s['Description']: s for s in webtris_sites }
    req_ids = [s['Id'] for s in webtris_sites]
    req_id_chunks = [req_ids[i:i + chunk_size] for i in range(0, len(req_ids), chunk_size)]
    print(len(req_id_chunks))
    for i in range(len(req_id_chunks)):
        print(f'Downloading chunk {i:03}...')
        chunk_data = get_chunk(req_id_chunks[i], date_from=date_from, date_to=date_to)
        joined_data = join_site_info(chunk_data, sites_dict)
        save_json_rows(joined_data, f'webtris_chunk_{date_from}_{date_to}_{i:03}')
        
# download_webtris_chunks(webtris_sites, '2019-07-01', '2019-07-31')
# download_webtris_chunks(webtris_sites, '2019-08-01', '2019-08-31')
# download_webtris_chunks(webtris_sites, '2019-09-01', '2019-09-30')
# download_webtris_chunks(webtris_sites, '2019-10-01', '2019-10-31')
# download_webtris_chunks(webtris_sites, '2019-11-01', '2019-11-30')
# download_webtris_chunks(webtris_sites, '2019-12-01', '2019-12-31')
download_webtris_chunks(webtris_sites, '2020-01-01', '2020-01-31')

213
https://webtris.highwaysengland.co.uk/api/v1/reports/01012020/to/31012020/Daily?sites=1,5,8,14,16,24,25,28,30,35,39,40,45,46,51,54,56,57,61,63&page=1&page_size=10000
{'row_count': 50592, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=1,5,8,14,16,24,25,28,30,35,39,40,45,46,51,54,56,57,61,63&start_date=01012020&end_date=31012020&page=2&page_size=10000', 'rel': 'nextPage'}]}
http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=1,5,8,14,16,24,25,28,30,35,39,40,45,46,51,54,56,57,61,63&start_date=01012020&end_date=31012020&page=2&page_size=10000
{'row_count': 50592, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=1,5,8,14,16,24,25,28,30,35,39,40,45,46,51,54,56,57,61,63&start_date=01012020&end_date=31012020&page=1&page_size=10000', 'rel': 'prevPage'}, {'href': 'http://webtris.nationalhighways.co.uk/api/v1

{'row_count': 56544, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=136,137,138,144,150,155,157,158,162,163,170,176,179,185,186,193,199,203,204,206&start_date=01012020&end_date=31012020&page=1&page_size=10000', 'rel': 'prevPage'}, {'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=136,137,138,144,150,155,157,158,162,163,170,176,179,185,186,193,199,203,204,206&start_date=01012020&end_date=31012020&page=3&page_size=10000', 'rel': 'nextPage'}]}
http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=136,137,138,144,150,155,157,158,162,163,170,176,179,185,186,193,199,203,204,206&start_date=01012020&end_date=31012020&page=3&page_size=10000
{'row_count': 56544, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=136,137,138,144,150,155,157,158,162,163,170,176,179,185,186,193,199,203,204,206&

{'row_count': 56544, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=253,256,259,265,269,279,288,289,290,296,301,303,304,308,309,314,315,317,319,325&start_date=01012020&end_date=31012020&page=2&page_size=10000', 'rel': 'prevPage'}, {'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=253,256,259,265,269,279,288,289,290,296,301,303,304,308,309,314,315,317,319,325&start_date=01012020&end_date=31012020&page=4&page_size=10000', 'rel': 'nextPage'}]}
http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=253,256,259,265,269,279,288,289,290,296,301,303,304,308,309,314,315,317,319,325&start_date=01012020&end_date=31012020&page=4&page_size=10000
{'row_count': 56544, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=253,256,259,265,269,279,288,289,290,296,301,303,304,308,309,314,315,317,319,325&

{'row_count': 56544, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=411,413,416,419,420,421,425,430,436,438,439,440,442,443,444,445,451,452,455,457&start_date=01012020&end_date=31012020&page=3&page_size=10000', 'rel': 'prevPage'}, {'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=411,413,416,419,420,421,425,430,436,438,439,440,442,443,444,445,451,452,455,457&start_date=01012020&end_date=31012020&page=5&page_size=10000', 'rel': 'nextPage'}]}
http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=411,413,416,419,420,421,425,430,436,438,439,440,442,443,444,445,451,452,455,457&start_date=01012020&end_date=31012020&page=5&page_size=10000
{'row_count': 56544, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=411,413,416,419,420,421,425,430,436,438,439,440,442,443,444,445,451,452,455,457&

{'row_count': 53568, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=504,508,511,523,526,533,538,542,544,545,547,549,551,555,556,562,565,566,570,572&start_date=01012020&end_date=31012020&page=4&page_size=10000', 'rel': 'prevPage'}, {'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=504,508,511,523,526,533,538,542,544,545,547,549,551,555,556,562,565,566,570,572&start_date=01012020&end_date=31012020&page=6&page_size=10000', 'rel': 'nextPage'}]}
http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=504,508,511,523,526,533,538,542,544,545,547,549,551,555,556,562,565,566,570,572&start_date=01012020&end_date=31012020&page=6&page_size=10000
{'row_count': 53568, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=504,508,511,523,526,533,538,542,544,545,547,549,551,555,556,562,565,566,570,572&

...

{'row_count': 59520, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=14476,14477,14478,14479,14480,14481,14482,14483,14484,14485,14486,14487,14488,14489,14490,14491,14492,14493,14494,14495&start_date=01012020&end_date=31012020&page=4&page_size=10000', 'rel': 'prevPage'}, {'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=14476,14477,14478,14479,14480,14481,14482,14483,14484,14485,14486,14487,14488,14489,14490,14491,14492,14493,14494,14495&start_date=01012020&end_date=31012020&page=6&page_size=10000', 'rel': 'nextPage'}]}
http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=14476,14477,14478,14479,14480,14481,14482,14483,14484,14485,14486,14487,14488,14489,14490,14491,14492,14493,14494,14495&start_date=01012020&end_date=31012020&page=6&page_size=10000
{'row_count': 59520, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhi

{'row_count': 59520, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=14516,14517,14518,14519,14520,14521,14522,14523,14524,14525,14526,14527,14528,14529,14530,14531,14532,14533,14534,14535&start_date=01012020&end_date=31012020&page=3&page_size=10000', 'rel': 'prevPage'}, {'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=14516,14517,14518,14519,14520,14521,14522,14523,14524,14525,14526,14527,14528,14529,14530,14531,14532,14533,14534,14535&start_date=01012020&end_date=31012020&page=5&page_size=10000', 'rel': 'nextPage'}]}
http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=14516,14517,14518,14519,14520,14521,14522,14523,14524,14525,14526,14527,14528,14529,14530,14531,14532,14533,14534,14535&start_date=01012020&end_date=31012020&page=5&page_size=10000
{'row_count': 59520, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhi

{'row_count': 59520, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=14556,14557,14558,14559,14560,14561,14562,14563,14564,14565,14566,14567,14568,14569,14570,14571,14572,14573,14574,14575&start_date=01012020&end_date=31012020&page=2&page_size=10000', 'rel': 'prevPage'}, {'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=14556,14557,14558,14559,14560,14561,14562,14563,14564,14565,14566,14567,14568,14569,14570,14571,14572,14573,14574,14575&start_date=01012020&end_date=31012020&page=4&page_size=10000', 'rel': 'nextPage'}]}
http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=14556,14557,14558,14559,14560,14561,14562,14563,14564,14565,14566,14567,14568,14569,14570,14571,14572,14573,14574,14575&start_date=01012020&end_date=31012020&page=4&page_size=10000
{'row_count': 59520, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhi

{'row_count': 20832, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=17152,17153,17154,17155,17156,17157,17158,17217,17218,17219,17220,17221,17223,17865,17879,18038,18039,18040,18041,18042&start_date=01012020&end_date=31012020&page=2&page_size=10000', 'rel': 'nextPage'}]}
http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=17152,17153,17154,17155,17156,17157,17158,17217,17218,17219,17220,17221,17223,17865,17879,18038,18039,18040,18041,18042&start_date=01012020&end_date=31012020&page=2&page_size=10000
{'row_count': 20832, 'start_date': '01012020', 'end_date': '31012020', 'links': [{'href': 'http://webtris.nationalhighways.co.uk/api/v1/reports/Daily?sites=17152,17153,17154,17155,17156,17157,17158,17217,17218,17219,17220,17221,17223,17865,17879,18038,18039,18040,18041,18042&start_date=01012020&end_date=31012020&page=1&page_size=10000', 'rel': 'prevPage'}, {'href': 'http://webtris.nationalhi

https://webtris.highwaysengland.co.uk/api/v1/reports/01012020/to/31012020/Daily?sites=18501,18504,18505,18508,18510,18511,18512,18515,18516,18520,18521,18524,18525,18542,18549,18550,18551,18552,18558,18563&page=1&page_size=10000
https://webtris.highwaysengland.co.uk/api/v1/reports/01012020/to/31012020/Daily?sites=18565,18566,18571,18572,18576,18580,18588,18593,18601,18602,18609,18619,18621,18622,18637,18648,18679,18680,18681,18682&page=1&page_size=10000
https://webtris.highwaysengland.co.uk/api/v1/reports/01012020/to/31012020/Daily?sites=18691,18695,18699,18703,18709,18712,18721,18741,18746,18762,18771,18772,18799,18801,18802,18818,18831,18833,18843,18845&page=1&page_size=10000
https://webtris.highwaysengland.co.uk/api/v1/reports/01012020/to/31012020/Daily?sites=18849,18967,18968,18969,18970,18971,18972,18973,18974,18975,18976,18977,18978,18979,18980,18981,18982,18983,18984,18985&page=1&page_size=10000
https://webtris.highwaysengland.co.uk/api/v1/reports/01012020/to/31012020/Daily?site

In [11]:
def t_from_time_bin(tb, do_floor=True):
    h, m = tb[10:].split(':')
    t = int(h) * 4 + int(float(m) / 15)
    if not do_floor:
        return t
    # T4c speed is floored, counters seem ceiled --> subtract 1
    t -= 1
    if t < 0:
        t = 95
    return t

def day_from_time_bin(tb, do_floor=True):
    if do_floor and t_from_time_bin(tb) == 95:
        d = datetime.strptime(tb, '%Y-%m-%d %H:%M') - timedelta(days=1)
        return d.strftime('%Y-%m-%d')
    return tb[:10]

def convert_webtris_chunks(do_floor=True):
    chunk_files = sorted((LONDON_PATH / 'downloads').glob('webtris_chunk_*.json'))
    df = pd.concat([pd.read_json(cf,  lines=True) for cf in chunk_files])
    df['id'] = df['id'].astype(str)
    df['heading'] = -1
    df['time_bin'] = [x[2] for x in df['time_bins']]
    df = df.rename(columns={'speed': 'speed_counter'})
    df['t'] = [t_from_time_bin(tb, do_floor) for tb in df['time_bin']]
    df['day'] = [day_from_time_bin(tb, do_floor) for tb in df['time_bin']]
    df = df[['id', 'name', 'lat', 'lon', 'heading', 'time_bin', 'volume', 'speed_counter', 't', 'day']]
    return df

webtris_df = convert_webtris_chunks()
webtris_df

,id,name,lat,lon,heading,time_bin,volume,speed_counter,t,day
0,14,A2/8392M,51.408466,0.381381,-1,2019-07-01 00:15,31,75.63898,0,2019-07-01
1,14,A2/8392M,51.408466,0.381381,-1,2019-07-01 00:30,17,72.42030,1,2019-07-01
2,14,A2/8392M,51.408466,0.381381,-1,2019-07-01 00:45,20,70.81096,2,2019-07-01
3,14,A2/8392M,51.408466,0.381381,-1,2019-07-01 01:00,17,77.24832,3,2019-07-01
4,14,A2/8392M,51.408466,0.381381,-1,2019-07-01 01:15,13,72.42030,4,2019-07-01
...,...,...,...,...,...,...,...,...,...,...
17851,18063,M5/8401A,51.508496,-2.660762,-1,2020-01-31 23:00,0,0.00000,91,2020-01-31
17852,18063,M5/8401A,51.508496,-2.660762,-1,2020-01-31 23:15,0,0.00000,92,2020-01-31
17853,18063,M5/8401A,51.508496,-2.660762,-1,2020-01-31 23:30,0,0.00000,93,2020-01-31
17854,18063,M5/8401A,51.508496,-2.660762,-1,2020-01-31 23:45,0,0.00000,94,2020-01-31


In [21]:
webtris_df['name'] = webtris_df['name'].astype(str)
webtris_df.to_parquet(LONDON_PATH / 'speed' / 'webtris_london_201907-202001.parquet', compression="snappy")

# Madrid

The raw files were downloaded from https://datos.madrid.es/ in zipped CSV files.

In [95]:
def download_dcat_zips(dcat_file, prefix, month_names=False, months=None):
    root = ET.parse(MADRID_PATH / 'downloads' / dcat_file).getroot()
    ns = {'dcat': 'http://www.w3.org/ns/dcat#',
          'dct': 'http://purl.org/dc/terms/'}
    data_urls = {}
    for entry in root.findall('.//dcat:Distribution', ns):
        title = entry.find('dct:title', ns).text
        url = entry.find('dcat:accessURL', ns).text
        if url.endswith('.zip'):
            try:
                if month_names:
                    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
                    title = title.replace('diciiembre', 'diciembre')
                    month = datetime.strptime(title, '%Y. %B')
                else:
                    month = datetime.strptime(title[6:], '%d/%m/%Y')
            except Exception as e:
                print(e)
                continue
            month = datetime.strftime(month, '%Y-%m')
            if months:
                if not month in months:
                    continue
            data_urls[month] = url
            print(f'{month}: {url}')
            r = requests.get(url, allow_redirects=True)
            open(MADRID_PATH / 'downloads' / f"{prefix}-{month}.zip", 'wb').write(r.content)
    return data_urls

In [87]:
url = "https://datos.madrid.es/egob/catalogo/202468-0-intensidad-trafico.dcat"
out_file = MADRID_PATH / 'downloads' / '202468-0-intensidad-trafico.dcat'
os.system(f"wget -O {out_file} {url}")

download_dcat_zips('202468-0-intensidad-trafico.dcat', prefix='locations', month_names=False,
                   months=['2021-06', '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12'])

2021-12: https://datos.madrid.es/egob/catalogo/202468-136-intensidad-trafico.zip
2021-11: https://datos.madrid.es/egob/catalogo/202468-133-intensidad-trafico.zip
2021-10: https://datos.madrid.es/egob/catalogo/202468-130-intensidad-trafico.zip
2021-09: https://datos.madrid.es/egob/catalogo/202468-125-intensidad-trafico.zip
2021-08: https://datos.madrid.es/egob/catalogo/202468-124-intensidad-trafico.zip
2021-07: https://datos.madrid.es/egob/catalogo/202468-121-intensidad-trafico.zip
2021-06: https://datos.madrid.es/egob/catalogo/202468-118-intensidad-trafico.zip


{'2021-12': 'https://datos.madrid.es/egob/catalogo/202468-136-intensidad-trafico.zip',
 '2021-11': 'https://datos.madrid.es/egob/catalogo/202468-133-intensidad-trafico.zip',
 '2021-10': 'https://datos.madrid.es/egob/catalogo/202468-130-intensidad-trafico.zip',
 '2021-09': 'https://datos.madrid.es/egob/catalogo/202468-125-intensidad-trafico.zip',
 '2021-08': 'https://datos.madrid.es/egob/catalogo/202468-124-intensidad-trafico.zip',
 '2021-07': 'https://datos.madrid.es/egob/catalogo/202468-121-intensidad-trafico.zip',
 '2021-06': 'https://datos.madrid.es/egob/catalogo/202468-118-intensidad-trafico.zip'}

In [96]:
url = "https://datos.madrid.es/egob/catalogo/208627-0-transporte-ptomedida-historico.dcat"
out_file = MADRID_PATH / 'downloads' / '208627-0-transporte-ptomedida-historico.dcat'
os.system(f"wget -O {out_file} {url}")

download_dcat_zips('208627-0-transporte-ptomedida-historico.dcat', prefix='data', month_names=True,
                   months=['2021-06', '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12'])

2021-12: https://datos.madrid.es/egob/catalogo/208627-103-transporte-ptomedida-historico.zip
2021-11: https://datos.madrid.es/egob/catalogo/208627-102-transporte-ptomedida-historico.zip
2021-10: https://datos.madrid.es/egob/catalogo/208627-101-transporte-ptomedida-historico.zip
2021-09: https://datos.madrid.es/egob/catalogo/208627-100-transporte-ptomedida-historico.zip
2021-08: https://datos.madrid.es/egob/catalogo/208627-99-transporte-ptomedida-historico.zip
2021-07: https://datos.madrid.es/egob/catalogo/208627-98-transporte-ptomedida-historico.zip
2021-06: https://datos.madrid.es/egob/catalogo/208627-97-transporte-ptomedida-historico.zip
time data '2014 año completo' does not match format '%Y. %B'
time data '2013 año completo' does not match format '%Y. %B'


{'2021-12': 'https://datos.madrid.es/egob/catalogo/208627-103-transporte-ptomedida-historico.zip',
 '2021-11': 'https://datos.madrid.es/egob/catalogo/208627-102-transporte-ptomedida-historico.zip',
 '2021-10': 'https://datos.madrid.es/egob/catalogo/208627-101-transporte-ptomedida-historico.zip',
 '2021-09': 'https://datos.madrid.es/egob/catalogo/208627-100-transporte-ptomedida-historico.zip',
 '2021-08': 'https://datos.madrid.es/egob/catalogo/208627-99-transporte-ptomedida-historico.zip',
 '2021-07': 'https://datos.madrid.es/egob/catalogo/208627-98-transporte-ptomedida-historico.zip',
 '2021-06': 'https://datos.madrid.es/egob/catalogo/208627-97-transporte-ptomedida-historico.zip'}

In [118]:
from math import atan2, cos, sin, degrees
from shapely.geometry import LineString, Point, MultiPoint
from shapely.ops import transform
from scipy.spatial import ConvexHull
import pyproj

ETRS89 = pyproj.CRS('EPSG:25830')
WGS84 = pyproj.CRS('EPSG:4326')
project_wgs84 = pyproj.Transformer.from_crs(ETRS89, WGS84, always_xy=True).transform

def get_heading(lon1, lat1, lon2, lat2):
    angle = atan2(lon2-lon1, lat2-lat1)
    angle = degrees(angle)
    if angle < 0:
        angle += 360
    return angle

def convert_locations_shp(zip_file):
    hdiff = 0
    features = []
    zf = zipfile.ZipFile(zip_file)
    with tempfile.TemporaryDirectory() as tempdir:
        zf.extractall(tempdir)
        shp_path = glob.glob(f'{tempdir}/*.shp')[0]
        print(shp_path)
        shapefile = geopandas.read_file(shp_path)
        for index, row in shapefile.iterrows():
            arrow_polygon = transform(project_wgs84, row['geometry'])
            from_point = LineString([arrow_polygon.exterior.coords[2], arrow_polygon.exterior.coords[3]]).centroid
            to_point = Point(arrow_polygon.exterior.coords[6])
            heading = get_heading(from_point.x, from_point.y, to_point.x, to_point.y)
            points = np.array(list(arrow_polygon.exterior.coords))
            hull_points = points[ConvexHull(points).vertices]
            # find shortest segment index
            shortest_segment_len = 1e9
            shortest_segment_idx = 0
            shortest_segment = None
            idxs = list(range(len(hull_points)))
            idxs.append(0)
            for i1, i2 in zip(idxs[:-1], idxs[1:]):
                s = LineString([hull_points[i1], hull_points[i2]])
                if s.length < shortest_segment_len:
                    shortest_segment_len = s.length
                    shortest_segment = s
                    shortest_segment_idx = i2
            from_point = shortest_segment.centroid
            to_idx = shortest_segment_idx
            for i in range(2):
                to_idx += 1
                if to_idx == len(hull_points):
                    to_idx = 0
            to_point = Point(hull_points[to_idx])
            heading_hull = get_heading(from_point.x, from_point.y, to_point.x, to_point.y)
            if abs(heading_hull - heading) > 0.5:
                hdiff += 1
                heading = heading_hull
            features.append(geojson.Feature(geometry=geojson.Point((row['longitud'], row['latitud'])),
            properties={
                "heading": heading, "name": row['nombre'], "tipo_elem": row['tipo_elem'],
                "id": int(row['id']), "distrito": str(row['distrito']), "cod_cent": str(row['cod_cent'])
            }))
    print(f'{hdiff} / {len(features)} headings differ')
    feature_collection = geojson.FeatureCollection(features)
    geojson_name = os.path.splitext(os.path.basename(zip_file))[0]
    with open(MADRID_PATH / 'downloads' / f'{geojson_name}.geojson', 'w') as f:
        geojson.dump(feature_collection, f)
    return feature_collection

def diff_dicts(a, b, drop_similar=True):
    res = a.copy()
    for k in res:
        if k not in b:
            res[k] = (res[k], None)
    for k in b:
        if k in res:
            res[k] = (res[k], b[k])
        else:
            res[k] = (None, b[k])
    if drop_similar:
        res = {k:v for k,v in res.items() if v[0] != v[1]}
    return res

def check_diff(diff, key, tolerance):
    if key not in diff:
        return False
    a, b = diff[key]
    return abs(a-b) > tolerance

def open_locations(locations_file):
    locations_by_id = {}
    with open(locations_file, 'r') as f:
        fc = geojson.load(f)
        for f in fc['features']:
            id = f['properties']['id']
            heading = f['properties']['heading']
            lat = f['geometry']['coordinates'][1]
            lon = f['geometry']['coordinates'][0]
            name = f['properties']['name']
            tipo_elem = f['properties']['tipo_elem']
            distrito = f['properties']['distrito']
            cod_cent = f['properties']['cod_cent']
            yield {"id": id, "name":name, "lat":lat, "lon":lon, "heading":heading,
                   "tipo_elem":tipo_elem, "distrito":distrito, "cod_cent":cod_cent}

def get_merged_locations():
    merged_locations = {}
    for locations_zip in (MADRID_PATH / 'downloads').glob('locations-*.zip'):
        convert_locations_shp(locations_zip)
        month = str(locations_zip)[-11:-4]
        print(month)
        for l in open_locations(str(locations_zip).replace('.zip', '.geojson')):
            id = l["id"]
            if id in merged_locations:
                matches = False
                for i, lo in enumerate(merged_locations[id]):
                    diff = diff_dicts(lo['data'], l)
                    if (check_diff(diff, 'lat', 0.0001) or 
                        check_diff(diff, 'lon', 0.0001) or 
                        check_diff(diff, 'heading', 5.0)):
                        pass
                    else:
                        matches = True
                        break
                if matches:
                    merged_locations[id][i]['months'].append(month)
                else:
                    merged_locations[id].append({'months': [month], 'data': l})
            else:
                merged_locations[id] = [{'months': [month], 'data': l}]
    return merged_locations


merged_locations = get_merged_locations()
len(merged_locations)

/var/folders/dr/jy4_0c5d3q37snprz6rhwwt40000gn/T/tmpyt1ch6td/pmed_ubicacion_07-2021.shp
502 / 4519 headings differ
2021-07
/var/folders/dr/jy4_0c5d3q37snprz6rhwwt40000gn/T/tmpcdzcftnv/pmed_ubicacion_12-2021.shp
502 / 4571 headings differ
2021-12
/var/folders/dr/jy4_0c5d3q37snprz6rhwwt40000gn/T/tmp7xl6mar8/pmed_ubicacion_06-2021.shp
502 / 4489 headings differ
2021-06
/var/folders/dr/jy4_0c5d3q37snprz6rhwwt40000gn/T/tmpe0afknj1/pmed_ubicacion_10-2021.shp
503 / 4538 headings differ
2021-10
/var/folders/dr/jy4_0c5d3q37snprz6rhwwt40000gn/T/tmpbt4egxk6/pmed_ubicacion_11-2021.shp
503 / 4567 headings differ
2021-11
/var/folders/dr/jy4_0c5d3q37snprz6rhwwt40000gn/T/tmpsakta8nm/pmed_ubicacion_08-2021.shp
502 / 4529 headings differ
2021-08
/var/folders/dr/jy4_0c5d3q37snprz6rhwwt40000gn/T/tmpnmu4tppr/pmed_ubicacion_09-2021.shp
502 / 4527 headings differ
2021-09


4593

In [120]:
features = []
for id, mll in merged_locations.items():
    for ml in mll:
        props = ml['data']
        props['valid_months'] = ','.join(ml['months'])
        pt = geojson.Point([props['lon'], props['lat']])
        features.append(geojson.Feature(geometry=pt, properties=props))

feature_collection = geojson.FeatureCollection(features)
with open(MADRID_PATH / 'downloads' / 'counter_locations_merged.geojson', 'w') as f:
    geojson.dump(feature_collection, f)

In [133]:
import csv
from io import TextIOWrapper

def time_bin_format(time):
    return time.strftime('%Y-%m-%d %H:%M')

def get_locations_by_id_and_month():
    locations_by_id_and_month = {}
    with open(MADRID_PATH / 'downloads' / 'counter_locations_merged.geojson', 'r') as f:
        fc = geojson.load(f)
        for f in fc['features']:
            id = f['properties']['id']
            heading = f['properties']['heading']
            lat = f['geometry']['coordinates'][1]
            lon = f['geometry']['coordinates'][0]
            for valid_month in f['properties']['valid_months'].split(','):
                if id not in locations_by_id_and_month:
                    locations_by_id_and_month[id] = {}
                locations_by_id_and_month[id][valid_month] = (lat, lon, heading)
    return locations_by_id_and_month

def generate_data_by_counter(month, locations_by_id_and_month):
    count = 0
    cf = MADRID_PATH / 'downloads'/ f'data-{month}.zip'
    print(cf)
    cfa = zipfile.ZipFile(cf, 'r')
    f = cfa.open(f'{(datetime.strptime(month, "%Y-%m")).strftime("%m-%Y")}.csv', 'r')
    csvreader = csv.reader(TextIOWrapper(f, 'utf-8'), delimiter=';')
    header = next(csvreader)
    # print(header)
    for row in csvreader:
        # https://datos.madrid.es/FWProjects/egob/Catalogo/Transporte/Trafico/ficheros/PuntosMedidaTraficoMdrid.pdf
        # ['id', 'fecha', 'tipo_elem', 'intensidad', 'ocupacion', 'carga', 'vmed', 'error', 'periodo_integracion']
        # fecha --> collection time --> time bin
        # tipo_elem --> counter type ('URB' or 'M30')
        # intensidad --> number of vehicles in 15 minutes --> volume
        # occupacion --> occupation time in percent [0..100] of 15 minutes
        # carga --> congestion level in percent [0..100]
        # vmed --> average speed (only on M30 counters)
        locid = int(row[0])
        if locid not in locations_by_id_and_month:
            print(f'WARNING: unknown ID {locid}')
            continue
        valid_month = month
        if valid_month not in locations_by_id_and_month[locid]:
            # Try the months before, stupid logic but sufficient here
            while valid_month not in locations_by_id_and_month[locid]:
                valid_month = (pd.to_datetime(valid_month) - pd.Timedelta("1 day")).strftime("%Y-%m")
                if valid_month[:4] == '2018':
                    break
            if valid_month not in locations_by_id_and_month[locid]:
                print(f'WARNING: no valid month {month} for ID {locid}: {locations_by_id_and_month[locid]}')
                continue
        lat, lon, heading = locations_by_id_and_month[locid][valid_month]
        volume = int(row[3])
        occupation = float(row[4])
        congestion_level = float(row[5])
        speed_avg = float(row[6])
        collection_time = row[1]
        time_bin = time_bin_format(datetime.strptime(collection_time, '%Y-%m-%d %H:%M:%S'))
        count += 1
        if count % 1000000 == 0:
            print(count)
        yield {'id': locid, 'lat': lat, 'lon': lon, 'heading': heading, 'time_bin': time_bin, 'type': row[2],
               'volume': volume, 'occupation': occupation, 'congestion_level': congestion_level,
               'speed_avg': speed_avg}

def process_madrid_month(month, output_path):
    locations_by_id_and_month = get_locations_by_id_and_month()
    month_df = pd.DataFrame(generate_data_by_counter(month, locations_by_id_and_month))
    month_df.to_parquet(output_path / 'all' / f'counters_{month}.parquet', compression="snappy")
    return month_df


# madrid_counters_df = process_madrid_month('2021-06', MADRID_PATH)
# madrid_counters_df

In [134]:
# process_madrid_month('2021-06', MADRID_PATH)
process_madrid_month('2021-07', MADRID_PATH)
# process_madrid_month('2021-08', MADRID_PATH)
# process_madrid_month('2021-09', MADRID_PATH)
# process_madrid_month('2021-10', MADRID_PATH)
# process_madrid_month('2021-11', MADRID_PATH)
# process_madrid_month('2021-12', MADRID_PATH)

/private/data/mets10/loop_counters/madrid/downloads/data-2021-07.zip
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000


,id,lat,lon,heading,time_bin,type,volume,occupation,congestion_level,speed_avg
0,1001,40.409729,-3.740786,62.428189,2021-07-01 00:00,M30,960,2.0,0.0,57.0
1,1001,40.409729,-3.740786,62.428189,2021-07-01 00:15,M30,936,2.0,0.0,62.0
2,1001,40.409729,-3.740786,62.428189,2021-07-01 00:30,M30,612,3.0,0.0,62.0
3,1001,40.409729,-3.740786,62.428189,2021-07-01 00:45,M30,324,1.0,0.0,58.0
4,1001,40.409729,-3.740786,62.428189,2021-07-01 01:00,M30,420,2.0,0.0,58.0
...,...,...,...,...,...,...,...,...,...,...
12132013,10845,40.468344,-3.647166,254.781625,2021-07-31 22:45,URB,121,1.0,4.0,0.0
12132014,10845,40.468344,-3.647166,254.781625,2021-07-31 23:00,URB,109,0.0,2.0,0.0
12132015,10845,40.468344,-3.647166,254.781625,2021-07-31 23:15,URB,134,1.0,3.0,0.0
12132016,10845,40.468344,-3.647166,254.781625,2021-07-31 23:30,URB,95,1.0,4.0,0.0
